# Data Preprocessing Tools

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importing dataset

Em um dataset temos features(variavel dependente) e variável independente(normalmente é a última coluna, é a variável que queremos predizer).

In [2]:
dataset = pd.read_csv('Data.csv')
X = dataset.iloc[:, :-1].values  # .values cria um array ao invés de um dataframe
y = dataset.iloc[:, -1]

In [3]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


In [4]:
print(y)

0     No
1    Yes
2     No
3     No
4    Yes
5    Yes
6     No
7    Yes
8     No
9    Yes
Name: Purchased, dtype: object


### Taking Care of Missing Data

Duas abordagens podem ser utilizadas para tratar missing data:
- Excluir dados nulos, se estes formarem pequenas quantidades diante do volume total de dados, não causando grandes distorções no resultado das análises.
- Substituir valores nulos pela média dos valores da coluna.

In [5]:
from sklearn.impute import SimpleImputer # módulo para tratar campos vazios

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 1:3])
X[:, 1:3] = imputer.transform(X[:, 1:3]) # retorn as colunas de 1 até 2 e substitui no array original 

In [6]:
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


# Encoding categorical data

## Encoding the Independent Variable

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# HOT enconde é uma maneira de criar 'ids' para categorias garantindo que não afetem o modelo de machine learning em relação à variável dependente
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = ct.fit_transform(X)

In [8]:
print(X)

[[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


## Encoding the Dependent Variable

In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [10]:
print(y)

[0 1 0 0 1 1 0 1 0 1]


# Splitting the dataset into the Training set and Test set

* Devemos fazer Feature Scaling antes ou depois de splitar o dataset em treino e teste?
R: Devemos fazer o Feature Scaling depois(essa é a única resposta correta), porque nosso conjunto de teste serve para validar/testar nosso modelo, representando dados reais, então não podemos fazer o scaling antes pois não seriam dados reais.

In [11]:
# Todos(ainda não sei se são todos mesmo) os modelos de machine learning esperam 4 datasets, 2 de teste(feature e dependent variable) e o mesmo vale para treino.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)   # recomendado 80% para treino e 20% para testes
# random_state = 1 é apenas teste.

In [12]:
print(X_train)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]


In [13]:
print(X_test)

[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]


In [14]:
print(y_train) # corresponde as 8 features em ordem do dataset(de features) para treino

[0 1 0 0 1 1 0 1]


In [15]:
print(y_test)

[0 1]


## Feature Scaling

Put all our features on the same scale
Obs: Não precisamos fazer feature scaling para todos os modelos de machine learning.

link: https://towardsdatascience.com/all-about-feature-scaling-bcc0ad75cb35

* **Standardisation:** É boa todo o tempo, faz seu trabalho sempre. Como funciona bem em todos os casos, é recomendado sempre começar por essa estratégia, pois vai fazer uma boa feature scaling e depois avaliar se suas features se encaixam para Normalisation.
* **Normalisation:** Recomendada quando temos uma distribuição normal na maioria das suas features.

In [16]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
# Pergunta muito frequente em machine learning: Temos que aplicar Feature Scaling em dados fictícios(Hot Encoded Feaure Categories)?
# R: Não, como no Standardisation os dados vão ser levados a escala de -3 até 3 mais ou menos, os dados hot encoded
# já estão dentro desse range e nada precisa ser feito nesse caso.
# Outro motivo para não faze-lo é que ao aplicar essa formula aos dados categóricos perdemos sua essencia, que é 
# representar uma categoria através de uma sequência, perdemos a informação de qual país é representado por qual sequência, no caso do dataset acima.
# Obs: Na aula o professor explica que em certos testes, aplicar feature scaling em variáveis categoricas(hot encoded) não trás ganhos significativos ao modelo.

X_train[:, 3:] = sc.fit_transform(X_train[:, 3:])
X_test[:, 3:] = sc.transform(X_test[:, 3:])
# Podemos aplicar metodos separados para standardisation: .fit e .transform, onde .fit calula a média e o desvio padrão e .transform aplica a fórmula de standardisation às features.
# Dessa forma podemos utilizar a mesma média e desvio padrão para fazer a Standardisation das features de teste.

# Porque temos que fazer o feature scaling do dataset de testes?
# Pois precisamos que o modelo seja congruente, precisamos ajustar o test set para a maneira que o modelo foi treinado.
# Precisamos aplicar o mesmo scaler (sc) que foi utilizado no X_train no X_test. 

In [17]:
print(X_train)

[[0.0 0.0 1.0 -0.19159184384578545 -1.0781259408412425]
 [0.0 1.0 0.0 -0.014117293757057777 -0.07013167641635372]
 [1.0 0.0 0.0 0.566708506533324 0.633562432710455]
 [0.0 0.0 1.0 -0.30453019390224867 -0.30786617274297867]
 [0.0 0.0 1.0 -1.9018011447007988 -1.420463615551582]
 [1.0 0.0 0.0 1.1475343068237058 1.232653363453549]
 [0.0 1.0 0.0 1.4379472069688968 1.5749910381638885]
 [1.0 0.0 0.0 -0.7401495441200351 -0.5646194287757332]]


In [18]:
print(X_test)

[[0.0 1.0 0.0 -1.4661817944830124 -0.9069571034860727]
 [1.0 0.0 0.0 -0.44973664397484414 0.2056403393225306]]
